In [ ]:
! pip install langchain_community tiktoken langchain-google-genai langchainhub chromadb langchain pypdf

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import chromadb
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.load import dumps, loads
from rpm_limiter import RPMLimiter, call_with_retry
from operator import itemgetter
import os

In [2]:
CREATE_DB=False

<h5>Load Document</h5>

In [3]:
loader = PyPDFLoader("R2vitamin.pdf")
docs = loader.load()

print(len(docs))

33


<h5>Chunking</h5>

In [4]:
if CREATE_DB:
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

    chunks = splitter.split_documents(docs)
    print("Total chunks:", len(chunks))

    for i, ch in enumerate(chunks[:3], start=1):
        print(f"\n==================== CHUNK {i} ====================")
        print(ch.page_content)

<h5>Vector database & Embedding</h5>

In [5]:
PERSIST_DIR ="./chroma_db"

os.makedirs(PERSIST_DIR, exist_ok=True)

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

client = chromadb.PersistentClient(path=PERSIST_DIR)

vectorstore = Chroma(
    client=client,
    collection_name="course_rag",
    embedding_function=embeddings,
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

C:\Users\teod4\AppData\Local\Temp\ipykernel_4500\183082865.py:9: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [6]:
if CREATE_DB:
    limiter = RPMLimiter(max_rpm=80)  

    batch_size = 60
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]
        call_with_retry(vectorstore.add_documents, batch, limiter=limiter)

<h1>Simple RAG</h1>

In [7]:
prompt = ChatPromptTemplate.from_template(
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, just say that you don't know. "
    "Use three sentences maximum and keep the answer concise.\n\n"
    "Question: {question}\n"
    "Context: {context}\n"
    "Answer:"
)

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What are fat-soluble vitamins and what do they do?")

"Fat-soluble vitamins include A, D, E, and K, which are absorbed with dietary fats and stored in the body's fatty tissues. They are vital for various functions such as vision, immune function, skin health, calcium absorption, bone health, and blood clotting. Vitamin E also acts as a powerful antioxidant, protecting cells from oxidative damage."

In [8]:
rag_chain.invoke("What is the capital of Japan?")

"I don't know the answer to this question based on the provided context. The context discusses various vitamins and their roles, as well as the industrial production of vitamin E. It does not contain any information about the capital of Japan."

<h1>Multi Query RAG</h1>

In [9]:
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives 
    | ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [10]:
def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]

retrieval_chain = generate_queries | retriever.map() | get_unique_union

In [11]:
template_mq ="""Use ONLY the following context to answer the question. If the context does not help the question say 'The context is not related'.

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template_mq)

mq_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
    
)


In [12]:
result = mq_rag_chain.invoke({"question": "What are fat-soluble vitamins and what do they do?"})
print(result)

C:\Users\teod4\AppData\Local\Temp\ipykernel_4500\1028870116.py:5: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


Fat-soluble vitamins are vitamins A, D, E, and K. They are absorbed with the help of dietary fats and are stored in the body's fatty tissues.

Their functions are:
*   **Vitamin A:** vital for vision, immune function, and skin health.
*   **Vitamin D:** essential for calcium absorption, bone health, and immune system regulation.
*   **Vitamin E:** acts as a powerful antioxidant, protecting cells from oxidative damage.
*   **Vitamin K:** plays a key role in blood clotting and bone metabolism.


In [13]:
result = mq_rag_chain.invoke({"question": "What is the capital of Japan?"})
print(result)

The context is not related.


<h1>HyDE RAG</h1>

In [18]:
template_hyde = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template_hyde)

generate_docs_for_retrieval = (
    prompt_hyde | llm | StrOutputParser() 
)

retrieval_chain = generate_docs_for_retrieval | retriever 

In [19]:
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

def qa_chain(query):
    retrieved_docs = retrieval_chain.invoke({"question":query})
    return final_rag_chain.invoke({"context":retrieved_docs,"question":query})

In [20]:
query = "What are fat-soluble vitamins and what do they do?"
result = qa_chain(query)
print(result)

Based on the provided context, Vitamin K is identified as a group of fat-soluble vitamins.

They play a crucial role in various physiological functions, particularly:
*   **Blood Clotting:** Essential for the process of blood clotting.
*   **Bone Metabolism:** Important for bone health.

There are two main forms:
*   **Vitamin K1 (phylloquinone):** Found in green leafy vegetables such as kale, spinach, broccoli, and Brussels sprouts.
*   **Vitamin K2 (menaquinone):** Found in fermented foods, certain animal products, and produced by bacteria in the human gut. Vitamin K2 is thought to be more bioavailable and has specific roles in extrahepatic tissues, including bones and blood vessels.
